![Redis](https://redis.com/wp-content/themes/wpx/assets/images/logo-redis.svg?auto=webp&quality=85,75&width=120)

# Introduction to Redis Python

This notebook introduces [Redis](https://redis.io) and the standard Python client, [redis-py](https://redis-py.readthedocs.io/en/stable/), for interacting with the database. We will explore the basics of Redis setup, data structures, and capabilities like vector search!

## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/financial-vss/blob/main/redis-py-01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Environment Setup

### Pull Github Materials
Because you are likely running this notebook in **Google Colab**, we need to first
pull the necessary dataset and materials directly from GitHub.

**If you are running this notebook locally**, FYI you may not need to perform this
step at all.

In [ ]:
# This clones the supporting git repository into a directory named 'temp_repo'.
!git clone https://github.com/redis-developer/financial-vss.git temp_repo

# This command moves the 'resources' directory from 'temp_repo' to your current directory.
!mv temp_repo/resources .
!mv temp_repo/requirements.txt .

# This deletes the 'temp_repo' directory, cleaning up the unwanted files.
!rm -rf temp_repo

### Install Python Dependencies

In [ ]:
!pip install -r requirements.txt

In [ ]:
import warnings

warnings.filterwarnings("ignore")

### Install Redis Stack

Later in this tutorial, Redis will be used to store, index, and query vector
embeddings created from PDF document chunks. **We need to make sure we have a Redis
instance available.**

#### Method 1: localized Redis Stack
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly
from the Redis package archive.

In [ ]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

#### Method 2: Redis Cloud
Instead of using the in-notebook, localized Redis Stack, you can quickly deploy a
[FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [ ]:
import os

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
#REDIS_HOST="redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
#REDIS_PORT=18374
#REDIS_PASSWORD="1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

## Hello World Redis

Now let's connect to the Redis db and get a basic feel for the most common
commands and data structures.

In [ ]:
import redis
import json
import numpy as np

from time import sleep

# Connect with the Redis Python Client
client = redis.Redis.from_url(REDIS_URL)

client.ping()

In [ ]:
client.dbsize() # should be empty

Redis, at it's core, is a simple key/value store. It supports a number of interesting
and flexible data structures that can solve a variatey of business and operational
problems.

### Strings

The basic string data type can be accessed using set/get methods. You can also place a
TTL policy (expiration) on any key in Redis.

In [ ]:
client.set("hello", "world")

In [ ]:
client.get("hello")

In [ ]:
client.delete("hello")

In [ ]:
client.set("hello", "world")
client.expire("hello", time=3)

sleep(4)

# should be EMPTY
client.get("hello")

### Hashes
Hashes are collections of key/value pairs that are grouped together. It gets
serialized as a string in Redis, but can hold a variety of data in each field.

You can think of a Hash as a one-level deep Python dictionary.


In [ ]:
obj = {
    "user": "john",
    "age": 45,
    "job": "dentist",
    "bio": "long form text of john's bio",
    "user_embedding": np.array([0.3, 0.4, -0.8], dtype=np.float32).tobytes() # cast vectors to bytes string
}

In [ ]:
client.hset("user:john", mapping=obj)

In [ ]:
client.hgetall("user:john")

In [ ]:
client.delete("user:john")

### JSON
With the JSON capabilitie enabled, Redis can be a drop-in replacement for MongoDB
or other slower document databases. You can store nested and structured JSON data
directly in Redis.

In [ ]:
# set a JSON obj
obj = {
    "user": "john",
    "metadata": {
        "age": 45,
        "job": "dentist",
    },
    "user_embedding": [0.3, 0.4, -0.8]
}

client.json().set("user:john", "$", obj)

In [ ]:
# get user JSON obj
client.json().get("user:john")

In [ ]:
# grab array length for embedding field
client.json().arrlen("user:john", "$.user_embedding")

In [ ]:
# grab obj keys
client.json().objkeys("user:john", "$")

In [ ]:
# delete user JSON
client.delete("user:john")

### Lists
Lists store sequences of information... potentially list of messages in an LLM
converstion flow, or really any list of items in a queue.

In [ ]:
# add items to a list
client.rpush("messages:john", *[
    json.dumps({"role": "user", "content": "Hello what can you do for me?"}),
    json.dumps({"role": "assistant", "content": "Hi, I am a helpful virtual assistant."})
])

In [ ]:
# list all items in the list using indices
[json.loads(msg) for msg in client.lrange("messages:john", 0, -1)]

In [ ]:
# count items in the list
client.llen("messages:john")

In [ ]:
# pop the first item from the list and push to another list
client.rpoplpush("messages:john", "read_messages:john")

In [ ]:
client.lrange("read_messages:john", 0, -1)

In [ ]:
# list cleanup
client.delete("messages:john", "read_messages:john")

### Pipelines
All Redis commands can be pipelined to gain some round trip latency improvements.

In [ ]:
with client.pipeline(transaction=False) as pipe:
    for i in range(50):
        pipe.json().set(f"user:{i}", "$", obj)
    # execute batch
    pipe.execute()

In [ ]:
client.dbsize()

In [ ]:
# clean up!
client.flushall()

## Intro to Vector Search in Redis

Now that we have the basics down, we will dive into the fundamentals of vector
search in Redis using the base Python client.

### Dataset Preparation (PDF Documents)

To best demonstrate Redis as a vector database layer, we will load a single
financial (10k filings) doc and preprocess it using some helpers from LangChain:

- `UnstructuredFileLoader` is not the only document loader type that LangChain provides. Docs: https://python.langchain.com/docs/integrations/document_loaders/unstructured_file
- `RecursiveCharacterTextSplitter` is what we use to create smaller chunks of text from the doc. Docs: https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader

# Load list of pdfs from a folder
data_path = "resources/"
docs = [os.path.join(data_path, file) for file in os.listdir(data_path)]

print("Listing available documents ...", docs)

In [ ]:
# fetch the Nike PDF
doc = [doc for doc in docs if "nke" in doc][0]

# set up the file loader/extractor and text splitter to create chunks
loader = UnstructuredFileLoader(doc, mode="single", strategy="fast")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2500, chunk_overlap=0)

# extract, load, and make chunks
chunks = loader.load_and_split(text_splitter)

print("Done preprocessing. Created", len(chunks), "chunks of the original pdf", doc)

In [ ]:
# Take a look at content from a chunk
print(chunks[25].page_content)

### Text embedding generation with SentenceTransformers

In [ ]:
from sentence_transformers import SentenceTransformer

# load model - may take a minute or two to download the first time
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
%%time

# create embeddings
chunk_embeddings = model.encode([chunk.page_content for chunk in chunks])
len(chunk_embeddings) == len(chunks)

### Set up some helper functions

Helper functions to encode the single query vector and display redis search results

In [ ]:
import pandas as pd


def encode_one(input):
    return model.encode(input).astype(np.float32).tobytes()


def table_view(res):
    if res.total == 0:
        print("No documents found.")
        return None
    
    res_df = pd.DataFrame([t.__dict__ for t in res.docs ]).drop(columns=["payload"])
    return res_df


### Define a schema and create an index
Below we connect to Redis and create an index for vector search that contains a single text field and vector field.

In [ ]:
from redis.commands.search.field import TagField, TextField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query


index_name = "redispy"
key_prefix = f"doc:{index_name}"


def create_index(index_type: str = "FLAT"):       # Creates a FLAT index by default
    try:
        # check to see if index exists
        client.ft(index_name).info()
        print("Index already exists!")
    except:
        # define schema
        schema = (
            TagField("doc_id"),                    # Tag Field - synthetic ID
            TextField("content"),                  # Text Field
            VectorField("chunk_vector",            # Vector Field
                index_type, {                      # Vector Index Type: FLAT or HNSW
                    "TYPE": "FLOAT32",
                    "DIM": 384,                    # Number of Vector Dimensions
                    "DISTANCE_METRIC": "COSINE",   # Vector Search Distance Metric
                }
            ),
        )

        # index Definition
        definition = IndexDefinition(prefix=[key_prefix], index_type=IndexType.HASH)

        # create Index
        client.ft(index_name).create_index(fields=schema, definition=definition)

In [ ]:
# Create the index
create_index()

In [ ]:
# Check the info related to the newly created index
client.ft(index_name).info()

### Process and load data using Redis
Below we use a Redis pipeline (not a transaction) to batch send writes to Redis. This method helps with throughput significantly. The batch_size param can be customized and benchmarked on your hardware and with your data. We typically recommend starting small (100-200) and increasing as needed.

In [ ]:
# load expects an iterable of dictionaries

batch_size = 200

with client.pipeline(transaction=False) as pipe:
    for i, chunk in enumerate(chunks):
        data = {
            'doc_id': f"{i}",
            'content': chunk.page_content,
            # For HASH -- must convert embeddings to bytes
            'chunk_vector': np.array(chunk_embeddings[i]).astype(np.float32).tobytes()
        }
        pipe.hset(f"{key_prefix}:{i}", mapping=data)
        # execute in "mini batches"
        if i % batch_size == 0:
            res = pipe.execute()

    # cleanup final batch execution
    res = pipe.execute()

In [ ]:
# check the data size in Redis
len(chunks) == client.dbsize()

In [ ]:
client.hgetall(f"{key_prefix}:0")

### Query the database
Now we can use the Redis search index to perform vector similarity search operations.

The code below takes a user input, converts to embeddings, and fetches the top 2 most semantically similar chunks from Redis.

In [ ]:
# Grab user input
_input = "Nike profit margins and company performance"

query = (
    Query("*=>[KNN 2 @chunk_vector $vector as vector_distance]")
     .sort_by("vector_distance")
     .return_fields("content", "vector_distance")
     .paging(0, 2)
     .dialect(2)
)

query_params = {
    "vector": encode_one(_input)
}

res = client.ft(index_name).search(query, query_params)

table_view(res)

In [ ]:
# Example of sorting by a field other than vector_distance
query = (
    Query("*=>[KNN 4 @chunk_vector $vector as vector_distance]")
     .sort_by("doc_id")
     .return_fields("doc_id", "content", "vector_distance")
     .paging(0, 4)
     .dialect(2)
)

query_params = {
    "vector": encode_one(_input)
}

res = client.ft(index_name).search(query, query_params)

table_view(res)

### Range Queries
Range queries allow you to set a pre defined "threshold" for which we want to return documents

In [ ]:
query = (
    Query("@chunk_vector:[VECTOR_RANGE $radius $vector]=>{$YIELD_DISTANCE_AS: vector_distance}")
     .sort_by("vector_distance")
     .return_fields("content", "vector_distance")
     .dialect(2)
)

# Find all vectors within 0.8 of the query vector
query_params = {
    "radius": 0.8,
    "vector": encode_one(_input)
}

res = client.ft(index_name).search(query, query_params)
table_view(res)

### Add filter statements
Redis queries can contain both vector search and traditional filters (numeric, tags, text, geo) in one single command.

In [ ]:
# filter for docs that contain "profit" in the content field and do KNN vector search
query = (
    Query("@content:profit=>[KNN 2 @chunk_vector $vector as vector_distance]")
     .sort_by("vector_distance")
     .return_fields("content", "vector_distance")
     .paging(0, 2)
     .dialect(2)
)

query_params = {
    "vector": encode_one(_input)
}

res = client.ft(index_name).search(query, query_params)
table_view(res)

In [ ]:
# lets clean up our index
client.ft(index_name).dropindex(True)

### What about JSON Support?

Redis also allows you to store data in JSON objects. The JSON fields can contain metadata and vectors. Below is a simple example of indexing JSON data.

In [ ]:
# schema
schema = (
    TextField("$.content",                     # Text Field (JSON path)
        as_name="content"                      # Text Field Alias -- required for JSON
    ),
    VectorField("$.chunk_vector",              # Vector Field (JSON path)
        "FLAT", {                              # Vector Index Type: FLAT or HNSW
            "TYPE": "FLOAT32",
            "DIM": 384,                        # Number of Vector Dimensions
            "DISTANCE_METRIC": "COSINE",       # Vector Search Distance Metric
        },
        as_name="chunk_vector"                 # Vector Field Alias -- required for JSON
    ),
)

# index Definition
definition = IndexDefinition(prefix=[key_prefix], index_type=IndexType.JSON) # select JSON here

# create Index
client.ft(index_name).create_index(fields=schema, definition=definition)

In [ ]:
client.ft(index_name).info()

In [ ]:
# Write JSON data to the index

batch_size = 200

with client.pipeline(transaction=False) as pipe:
    for i, chunk in enumerate(chunks):
        redis_key = f"{key_prefix}:{i}"
        data = {
            'content': chunk.page_content,
            'chunk_vector': chunk_embeddings[i].tolist() # notice that we don't need to convert JSON embeddings to bytes
        }
        #print(data)
        pipe.json().set(redis_key, "$", data)
        # mini batch
        if i % batch_size == 0:
            res = pipe.execute()

    res = pipe.execute() # make sure to use mini batches if working with larger datasets

In [ ]:
# Fetch the JSON doc
client.json().get(f"{key_prefix}:0", "$")

In [ ]:
# And now you can perform the same kinds of queries
query = (
    Query("@content:profit=>[KNN 2 @chunk_vector $vector as vector_distance]")
     .sort_by("vector_distance")
     .return_fields("content", "vector_distance")
     .paging(0, 2)
     .dialect(2)
)

query_params = {
    "vector": encode_one(_input)

}
res = client.ft(index_name).search(query, query_params)

table_view(res)

## Cleanup
Clean up the index and data.

In [ ]:
client.ft(index_name).dropindex(True)

## What's Next?

Now that you have the basics down with the baseline Redis Python client:

| Notebook | Description | Documentation |
|----------|-------------|---------------|
| [**redisvl-02**](https://colab.research.google.com/github/redis-developer/financial-vss/blob/main/redisvl-02.ipynb) | Dive deeper using a dedicated VSS client library to build a RAG application from scratch including some advanced techniques using an OpenAI LLM. | [View Docs](https://redisvl.com) |
| [**langchain-03**](https://colab.research.google.com/github/redis-developer/financial-vss/blob/main/langchain-03.ipynb) | Wrap up with an integrated approach via LangChain and see how to build complex LLM chains. | [View Docs](https://python.langchain.com/docs/get_started/introduction) |
